# Red neuronal convolutiva: Dataset Boats

## Optimización de hiperparámetros con Hyperas

Para la segunda parte de esta práctica, vamos a optimizar los valores de los hiperparámetros utilizando un wrapper de hyperopt llamado [Hyperas](https://github.com/maxpumperla/hyperas). Esta libreria, nos permite, de una forma sencilla obtener el mejor valor de los hiperparámetros mediante la utilización del método *minimize*.

Primero vamos a montar el directorio de Drive en nuetro notebook.

In [7]:
# Conexión con Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Como esta librería no viene incluida la instalamos:

In [8]:
!pip install hyperas

### Optimización de hiperparámetros

Ahora que ya tenemos todo preparado, vamos a optimizar los valores de los hiperparámetros de nuestro modelo.

Antes que nada, necesitamos crear nuestro modelo y nuestro dataset, que serán utilizados por el optimizador para calcular los mejores parámetros. Estos son creados a partir de los métodos *model_designer()* y *data_designer()*, respectivamente.

*Data_designer* crea dos *ImageDataGenerator* —uno de entrenamiento y otro para test—. Estos son devueltos en el mismo orden en el que los toma la función *model_designer*. Esto es importante, pues la función  *minimize* de la clase *optim* del módulo *hyperas* utiliza la salida de la función que crea los datos para introducirlo como parámetros para la función que crea el modelo.

*Model_designer* crea el modelo y realiza el entrenamiento de una evaluación del optimizador de hiperparámetros. Una vez realizado el entrenamiento, se devuelve el loss y el modelo creado en ese entrenamiento. El valor de 'loss' es utilizado por el optimizador para minimizar su valor. 

La elección de los valores de hiperparámetros puede indicarse mediante algún tipo de distribución (las funciones son aportadas por el módulo *hyperas.distributions*). Para este ejemplo, utilizaremos las distribuciones: *choice*, con el que seleccionamos entre valores discretos; y *uniform*, que permite calcular un valor aleatorio entre dos valores reales.


Una vez creado las funciones *data_designer* y *model_designer*, las podemos utilizar para realizar la optimización con la función *minimize*, la cual requiere dos funciones que crean el conjunto de datos y otro que crea y devuelve el valor de un entrenamiento para un cierto valor de hiperparámetros, dentro del rango especificado.

Además, podemos elegir un algoritmo para realizar la optimización con el parámetro *algo* —tpe.suggest o rand.suggest—, el número de intento que queremos realizar y un objeto trials —donde obtendremos posteriormente el valor de cada evaluacion y el modelo—.

In [13]:
#from __future__ import print_function

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import keras


from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def data_designer():
  train_data_dir = '/content/drive/My Drive/Colaboratory/Practica2/dataset/boats/TRAIN'
  validation_data_dir = '/content/drive/My Drive/Colaboratory/Practica2/dataset/boats/TEST'

  train_datagen = ImageDataGenerator(
          rescale=1./255, # Factor de reescalado
          shear_range=0.2, # Giro de imagen
          zoom_range=0.2,
          horizontal_flip=True)

  test_datagen = ImageDataGenerator(rescale=1./255)

  batch_size_train = 20
  batch_size_validate = 32

  train_generator = train_datagen.flow_from_directory(
          train_data_dir,
          target_size=(200, 200), # Reescalado de la imagen a tamaño igualado
          batch_size=batch_size_train, # Ajuste del tamaño de los batches
          class_mode='categorical')

  validation_generator = test_datagen.flow_from_directory(
          validation_data_dir,
          target_size=(200, 200),
          batch_size=batch_size_validate,
          class_mode='categorical')
  return train_generator, validation_generator


def model_designer(train_generator, validation_generator):
  
  
  model = Sequential()
  model.add(Conv2D({{choice([16, 32, 64])}}, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(200, 200, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D({{choice([16, 32, 64])}}, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout({{uniform(0, 0.5)}}))
  model.add(Flatten())
  model.add(Dense({{choice([64, 128, 256])}}, activation='relu'))
  model.add(Dropout({{uniform(0, 0.5)}}))
  model.add(Dense(9, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
  
  epochs = 10 #10

  hist = model.fit_generator(
          train_generator,
          steps_per_epoch=10, #10
          epochs=epochs, 
          validation_data=validation_generator,
          validation_steps=80, #80
  )
  ev = model.evaluate_generator(validation_generator, steps=1)
  return {'loss': ev[0], 'status': STATUS_OK, 'model': model}

print("Start optimization: ")
trials = Trials()
best_run, best_model = optim.minimize(model=model_designer,
                                          data=data_designer,
                                          algo=tpe.suggest,
                                          max_evals=7, # 7
                                          trials=trials,
                                          eval_space=True,
                                          return_space=False,
                                          notebook_name='drive/My Drive/Colaboratory/Practica2/BoatsHyperas')


Start optimization: 
>>> Imports:
#coding=utf-8

try:
    from google.colab import drive
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout
except:
    pass

try:
    from keras.optimizers import RMSprop
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import keras
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Conv2D': hp.choice('Conv2D', [16, 32, 64]),
        'Conv2D_1': hp.choice('Conv2D_1', [16, 32, 64]),


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.117343). Check your callbacks.
  % delta_t_median)




 5/10 [==============>...............]
 - ETA: 0s - loss: 1.7798 - acc: 0.3300

 6/10 [=================>............]
 - ETA: 0s - loss: 1.8236 - acc: 0.3000

 7/10 [====================>.........]
 - ETA: 0s - loss: 1.8050 - acc: 0.3071

 8/10 [=======================>......]
 - ETA: 0s - loss: 1.7623 - acc: 0.3250

 9/10 [==========================>...]
 - ETA: 0s - loss: 1.7810 - acc: 0.3278

10/10 [==============================]
 - 49s 5s/step - loss: 1.7704 - acc: 0.3350 - val_loss: 1.6974 - val_acc: 0.4077

Epoch 6/10
 1/10

### Resultados de la optimización

Ya tenemos el resultado de la optimización. Los resultados más revelantes los encontraremos en *best_run*, que nos devuelve un diccionario con los mejores valores de hiperparámetros encontrados; y en *Trials* que, entre otros resultados, en la lista *results* nos devuelve un diccionario con el valor 'loss' de cada evaluación, el estado de la evaluación y el modelo creado (este diccionario es el que devolvíamos en *model_designer*).

In [14]:
train_generator, validation_generator = data_designer()
ev = best_model.evaluate_generator(validation_generator, steps=1)

print("\n---------- Best run ----------\n")
print("Loss: " + str(ev[0]) + " Acc: " + str(ev[1]) + "\n\n")

print("Fitted hyperparameters: \n")

print("    First Conv2D: " + str(best_run['Conv2D']) + "\n")
print("    Second Conv2D: " + str(best_run['Conv2D_1']) + "\n")
print("    Dense: " + str(best_run['Dense']) + "\n")
print("    First dropout: " + str(best_run['Dropout']) + "\n")
print("    Second dropout: " + str(best_run['Dropout']) + "\n")


print("\n---------- Trials ----------\n")
for n, result in enumerate(trials.results):
  print("Trial " + str(n+1) + " -> " + str(result) + "\n")



Found 1095 images belonging to 9 classes.
Found 368 images belonging to 9 classes.

---------- Best run ----------

Loss: 1.649768352508545 Acc: 0.34375


Fitted hyperparameters: 

    First Conv2D: 64

    Second Conv2D: 32

    Dense: 256

    First dropout: 0.45641472349028517

    Second dropout: 0.45641472349028517


---------- Trials ----------

Trial 1 -> {'loss': 1.7581501007080078, 'status': 'ok', 'model': <keras.engine.sequential.Sequential object at 0x7fae1bc62f28>}

Trial 2 -> {'loss': 1.496720552444458, 'status': 'ok', 'model': <keras.engine.sequential.Sequential object at 0x7fae1b83ccf8>}

Trial 3 -> {'loss': 1.582949161529541, 'status': 'ok', 'model': <keras.engine.sequential.Sequential object at 0x7fae1c4dbd30>}

Trial 4 -> {'loss': 1.894469141960144, 'status': 'ok', 'model': <keras.engine.sequential.Sequential object at 0x7fae194b7f60>}

Trial 5 -> {'loss': 1.7421374320983887, 'status': 'ok', 'model': <keras.engine.sequential.Sequential object at 0x7fae176900b8>}

Tria